In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import research_agents.arxiv_download as adl
import research_agents.html_to_ref_list as h2rl
import research_agents.summarize as summ
import anthropic
from IPython.display import display, HTML
from html import escape
from research_agents.get_relevant_papers import get_relevant_papers
from research_agents.prophy_scrape import download_prophy


arxiv_url = 'https://arxiv.org/abs/2004.04136'
# stop_len = 'Number of comparison papers to filter'

display(HTML("<h2>Thinking...</h2>"))

# Declare client 
api_file = open("api_key.txt", "r")
api_key = api_file.read()
client = anthropic.Client(api_key)

# Arxiv URL to html
display(HTML(f'<p>Downloading <a href="{escape(arxiv_url)}" target=_blank><code>{escape(arxiv_url)}</code></a></p>'))
html = adl.download_arxiv_html(arxiv_url)

# Prompt prophy for candidate reference papers (title + authors + abstract)
title_original, abstract_original, citing_data = download_prophy(arxiv_url)
title_refs = [d['title'] for d in citing_data]
abstract_refs = [d['abstract'] for d in citing_data]
arxivids_refs = [d['arxiv'] for d in citing_data]

# Filter out papers that are not relevant
stop_len = 3
good_idxs = get_relevant_papers(client, title_original, abstract_original, title_refs, abstract_refs, stop_len)
title_refs = [title_refs[i] for i in good_idxs]
abstract_refs = [abstract_refs[i] for i in good_idxs]
arxivids_refs = [arxivids_refs[i] for i in good_idxs]

print(f'Number of relevant follow-ups: {len(title_refs)}')

responses = []
for i, (ref_title, ref_abstract) in enumerate(zip(title_refs, abstract_refs)):
    # For each relevant paper, extract snippets that mention the original paper

    # print(f'Reference title: {ref_title}')
    ref_html = adl.download_arxiv_html(arxivids_refs[i])

    responses.append(summ.summarize_differences(client, html, ref_html))

# Summarize all the summaries in a single paragraph
meta_summary = summ.make_meta_summary(client=client, responses=responses, title_original=title_original)

print(meta_summary)